# Analysis of S3 objects using Python

**Data Set**
[Kagggle Financial Data Set](https://www.kaggle.com/borismarjanovic/price-volume-data-for-all-us-stocks-etfs/downloads/price-volume-data-for-all-us-stocks-etfs.zip/3)
_Note:_ Only ""aapl" and "ge" uploaded

This demo shows the following:
* List objects in an S3 bucket
* Use python Boto3 package to programmatically connect to S3 and process objects
* Use pandas to calculate the size of the folders
* Load the files into dataframe

------------
## List the folders in s3 bucket

!aws s3 ls --profile fin-demo
2019-09-13 11:16:34 rsdg-fin-demo-price-eu-west-2
2019-09-13 11:16:34 rsdg-fin-demo-reference-eu-west-2
2019-09-13 11:16:34 rsdg-fin-demo-transaction-eu-west-2
2019-09-12 23:18:23 rsdg-s3-bucket-fin-demo

****
## List the files in the **rsdg-s3-bucket-fin-demo**

In [1]:
!aws s3 ls s3://rsdg-s3-bucket-fin-demo/ --profile fin-demo

2019-09-12 23:18:25     428286 aapl.us.txt
2019-09-12 23:18:25     705515 ge.us.txt


****
## Use boto3 package to query the s3 bucket 

In [2]:
import boto3

_session = boto3.Session(profile_name='fin-demo')
s3 = _session.client('s3')
s3.list_objects_v2(Bucket='rsdg-s3-bucket-fin-demo')

{'ResponseMetadata': {'RequestId': 'E4329C18BC3A2C07',
  'HostId': 'imPxZotDfw9igaLxSRGcuTMcSItx3ri0IuiNI2nvpjCNO8kzzSP/5fiL4XrzCRYRGetgPEt8hHQ=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'imPxZotDfw9igaLxSRGcuTMcSItx3ri0IuiNI2nvpjCNO8kzzSP/5fiL4XrzCRYRGetgPEt8hHQ=',
   'x-amz-request-id': 'E4329C18BC3A2C07',
   'date': 'Fri, 13 Sep 2019 10:38:29 GMT',
   'x-amz-bucket-region': 'eu-west-2',
   'content-type': 'application/xml',
   'transfer-encoding': 'chunked',
   'server': 'AmazonS3'},
  'RetryAttempts': 0},
 'IsTruncated': False,
 'Contents': [{'Key': 'aapl.us.txt',
   'LastModified': datetime.datetime(2019, 9, 12, 22, 18, 25, tzinfo=tzutc()),
   'ETag': '"fb1a8522d092533204e9c49bf6aef44f"',
   'Size': 428286,
   'StorageClass': 'STANDARD'},
  {'Key': 'ge.us.txt',
   'LastModified': datetime.datetime(2019, 9, 12, 22, 18, 25, tzinfo=tzutc()),
   'ETag': '"8687f5de7acb53fc802b12d1fc4fd3fa"',
   'Size': 705515,
   'StorageClass': 'STANDARD'}],
 'Name': 'rsdg-s3-bucket-f

****
## Process boto3 json to build a dictionary of files and objects

In [3]:
def get_s3_keys(bucket):
    """Get a list of keys in an S3 bucket."""
    _keys = {}
    resp = s3.list_objects_v2(Bucket=bucket)
    for obj in resp['Contents']:
        _key = obj['Key']
        _size = obj['Size']
        _keys[_key] = _size
    return _keys

keysAndSizes = {}
keysAndSizes = get_s3_keys('rsdg-s3-bucket-fin-demo')
print(keysAndSizes)

{'aapl.us.txt': 428286, 'ge.us.txt': 705515}


****
## Load the dictionary into a pandas dataframe

In [4]:
import pandas as pd
pdKeysAndSizes = pd.DataFrame(list(keysAndSizes.items()))
pdKeysAndSizes.columns = ['FileName', 'Size']
pdKeysAndSizes

,FileName,Size
0,aapl.us.txt,428286
1,ge.us.txt,705515


****
## Use pandas to count the number of files and total size of files

In [5]:
print("Number of files: %s; Total size of files (MB): %s" % (pdKeysAndSizes.count()['FileName'],
                                                        pdKeysAndSizes.sum(axis=0)['Size']/1024/1024
                                                       ))

Number of files: 2; Total size of files (MB): 1.0812768936157227


## Sample the file to understand the structure

In [6]:
import smart_open as so
ii = 0
for line in so.open('s3://rsdg-s3-bucket-fin-demo/aapl.us.txt', transport_params=dict(session= _session) ):
    ii += 1
    if(ii < 10):
        print(line)
    else:
        break

Date,Open,High,Low,Close,Volume,OpenInt

1984-09-07,0.42388,0.42902,0.41874,0.42388,23220030,0

1984-09-10,0.42388,0.42516,0.41366,0.42134,18022532,0

1984-09-11,0.42516,0.43668,0.42516,0.42902,42498199,0

1984-09-12,0.42902,0.43157,0.41618,0.41618,37125801,0

1984-09-13,0.43927,0.44052,0.43927,0.43927,57822062,0

1984-09-14,0.44052,0.45589,0.44052,0.44566,68847968,0

1984-09-17,0.45718,0.46357,0.45718,0.45718,53755262,0

1984-09-18,0.45718,0.46103,0.44052,0.44052,27136886,0



## Load "aapl" file into data frame

In [7]:

equity_df = pd.read_csv(so.open('s3://rsdg-s3-bucket-fin-demo/aapl.us.txt', transport_params=dict(session= _session) ))
equity_df['Stock']='aapl.us.txt'.replace('.txt','')
equity_df

,Date,Open,High,Low,Close,Volume,OpenInt,Stock
0,1984-09-07,0.42388,0.42902,0.41874,0.42388,23220030,0,aapl.us
1,1984-09-10,0.42388,0.42516,0.41366,0.42134,18022532,0,aapl.us
2,1984-09-11,0.42516,0.43668,0.42516,0.42902,42498199,0,aapl.us
3,1984-09-12,0.42902,0.43157,0.41618,0.41618,37125801,0,aapl.us
4,1984-09-13,0.43927,0.44052,0.43927,0.43927,57822062,0,aapl.us
...,...,...,...,...,...,...,...,...
8359,2017-11-06,171.75000,174.36000,171.10000,173.63000,34901241,0,aapl.us
8360,2017-11-07,173.29000,174.51000,173.29000,174.18000,24424877,0,aapl.us
8361,2017-11-08,174.03000,175.61000,173.71000,175.61000,24451166,0,aapl.us
8362,2017-11-09,174.48000,175.46000,172.52000,175.25000,29533086,0,aapl.us


## Load the files into data frame

In [8]:
combined_equity_df = pd.DataFrame()

for key in keysAndSizes:
    file = 's3://rsdg-s3-bucket-fin-demo/' + key
    single_equity_df = pd.DataFrame()
    single_equity_df = pd.read_csv(so.open(file, transport_params=dict(session= _session) ))
    single_equity_df['Stock'] = key.replace('.txt','')
    combined_equity_df = combined_equity_df.append(single_equity_df, ignore_index=True)
        
print(combined_equity_df)

             Date      Open      High       Low     Close     Volume  OpenInt  \
0      1984-09-07   0.42388   0.42902   0.41874   0.42388   23220030        0   
1      1984-09-10   0.42388   0.42516   0.41366   0.42134   18022532        0   
2      1984-09-11   0.42516   0.43668   0.42516   0.42902   42498199        0   
3      1984-09-12   0.42902   0.43157   0.41618   0.41618   37125801        0   
4      1984-09-13   0.43927   0.44052   0.43927   0.43927   57822062        0   
...           ...       ...       ...       ...       ...        ...      ...   
22417  2017-11-06  20.52000  20.53000  20.08000  20.13000   60641787        0   
22418  2017-11-07  20.17000  20.25000  20.12000  20.21000   41622851        0   
22419  2017-11-08  20.21000  20.32000  20.07000  20.12000   39672190        0   
22420  2017-11-09  20.04000  20.07100  19.85000  19.99000   50831779        0   
22421  2017-11-10  19.98000  20.68000  19.90000  20.49000  100698474        0   

         Stock  
0      aap

### Related notebooks
* [00-Setup](./00_setup.ipynb)
* [01-Process S3 using python](./01_Process_s3_files.ipynb)
* [02-Visualization and Analytics](./02_Visualization_and_Analytics.ipynb)
* [03-Risk Analytics](./03_Risk_Analytics.ipynb)
* [04-Exploring Firehose,Athena and Quicksight](./04_Exploring_Kinesis_Firehose.ipynb)
* [05-Athena and Quicksights](./05_Athena_Quicksight.ipynb)
* [06-Sagemaker to run the notebooks](./06_Sagemaker_jupyterlab.ipynb)
* [07_Transform stream data using Lambda](./07_Transform_lambda.ipynb)
* [08_Move data to Redshift using Glue](./08_Glue_Redshift.ipynb)
* [09_CI/CD Terrform with Travis CI](./09_Integrating_terraform_travisci.ipynb)